In [11]:
import java.io.File
import org.apache.spark.sql.types.{StructField, StructType, StringType, IntegerType}
import org.apache.spark.sql.DataFrame

val dataDir = "CNASeg"
val ctcFile = "CTCgains_kanwar2015.csv"
val ctcSchema = new StructType(Array(
                    new StructField("ctcChr", IntegerType, false),
                    new StructField("ctcStart", IntegerType, false),
                    new StructField("ctcEnd", IntegerType, false),
                    new StructField("sizeBp", StringType, false),
                    new StructField("sizeKbp", StringType, false)))
val ctcDf = spark.read.format("csv").option("sep",",").option("header", true).schema(ctcSchema).load(ctcFile)

import java.io.File
import org.apache.spark.sql.types.{StructField, StructType, StringType, IntegerType}
import org.apache.spark.sql.DataFrame
dataDir: String = CNASeg
ctcFile: String = CTCgains_kanwar2015.csv
ctcSchema: org.apache.spark.sql.types.StructType = StructType(StructField(ctcChr,IntegerType,false),StructField(ctcStart,IntegerType,false),StructField(ctcEnd,IntegerType,false),StructField(sizeBp,StringType,false),StructField(sizeKbp,StringType,false))
ctcDf: org.apache.spark.sql.DataFrame = [ctcChr: int, ctcStart: int ... 3 more fields]


In [12]:
val pattern = """([0-9]+)_fin\.cna\.seg""".r
val files: Array[String] = new File(dataDir).listFiles.map(f => f.getName())
val dataFrames: Array[DataFrame] = new Array(28)
var i: Int = 0
for (f <- files) {
    val pattern(num) = f
    val segSchema = new StructType(Array(
                    new StructField("chr", IntegerType, false),
                    new StructField("start", IntegerType, false),
                    new StructField("end", IntegerType, false),
                    new StructField(s"${num}_fin.copy.number", StringType, false),
                    new StructField(s"${num}_fin.event", StringType, false),
                    new StructField(s"${num}_fin.logR", StringType, false),
                    new StructField(s"${num}_fin.subclone.status", StringType, false),
                    new StructField(s"${num}_fin.Corrected_Copy_Number", StringType, false),
                    new StructField(s"${num}_fin.Corrected_Call", StringType, false),
                    new StructField(s"${num}_fin.logR_Copy_Number", StringType, false)))
    val df = spark.read.format("csv")
            .option("sep","\t")
            .option("header", true)
            .schema(segSchema)
            .load(s"$dataDir/$f")
            .withColumn("sampleNumber", lit(num))
            .withColumnRenamed(s"${num}_fin.copy.number", "copyNumber")
            .withColumnRenamed(s"${num}_fin.event", "event")
            .withColumnRenamed(s"${num}_fin.logR", "logR")
            .withColumnRenamed(s"${num}_fin.subclone.status", "subcloneStatus")
            .withColumnRenamed(s"${num}_fin.Corrected_Copy_Number", "CorrectedCopyNumber")
            .withColumnRenamed(s"${num}_fin.Corrected_Call", "correctedCall")
            .withColumnRenamed(s"${num}_fin.logR_Copy_Number", "logRCopyNumber")
            .filter(col("correctedCall") === "GAIN" || col("correctedCall") === "AMP" || col("correctedCall") === "HLAMP")
    dataFrames(i) = df
    i+=1
}

pattern: scala.util.matching.Regex = ([0-9]+)_fin\.cna\.seg
files: Array[String] = Array(35_fin.cna.seg, 36_fin.cna.seg, 46_fin.cna.seg, 26_fin.cna.seg, 18_fin.cna.seg, 15_fin.cna.seg, 24_fin.cna.seg, 30_fin.cna.seg, 32_fin.cna.seg, 29_fin.cna.seg, 12_fin.cna.seg, 53_fin.cna.seg, 5_fin.cna.seg, 44_fin.cna.seg, 17_fin.cna.seg, 21_fin.cna.seg, 14_fin.cna.seg, 50_fin.cna.seg, 3_fin.cna.seg, 27_fin.cna.seg, 11_fin.cna.seg, 8_fin.cna.seg, 33_fin.cna.seg, 20_fin.cna.seg, 2_fin.cna.seg, 9_fin.cna.seg, 22_fin.cna.seg, 47_fin.cna.seg)
dataFrames: Array[org.apache.spark.sql.DataFrame] = Array([chr: int, start: int ... 9 more fields], [chr: int, start: int ... 9 more fields], [chr: int, start: int ... 9 more fields], [chr: int, start: int ... 9 more fields], [chr: int, start: int ... 9 more fields...


In [13]:
var df: DataFrame = dataFrames(0)
(1 to 27).foreach(i => df = df.union(dataFrames(i)))
df.count

df: org.apache.spark.sql.DataFrame = [chr: int, start: int ... 9 more fields]
res10: Long = 20250


In [14]:
df.printSchema

root
 |-- chr: integer (nullable = true)
 |-- start: integer (nullable = true)
 |-- end: integer (nullable = true)
 |-- copyNumber: string (nullable = true)
 |-- event: string (nullable = true)
 |-- logR: string (nullable = true)
 |-- subcloneStatus: string (nullable = true)
 |-- CorrectedCopyNumber: string (nullable = true)
 |-- correctedCall: string (nullable = true)
 |-- logRCopyNumber: string (nullable = true)
 |-- sampleNumber: string (nullable = false)



In [15]:
ctcDf.show

+------+---------+---------+-------+------------------+
|ctcChr| ctcStart|   ctcEnd| sizeBp|           sizeKbp|
+------+---------+---------+-------+------------------+
|     1|  1828508|  3571573|1743065|1743.0650000000001|
|     1| 16122977| 16173900|  50923|50.923000000000002|
|     1|155247948|156217829| 969881|969.88099999999997|
|     1|154963903|155224815| 260912|260.91199999999998|
|     7| 73115776| 75603639|2487863|2487.8629999999998|
|     7| 99728602|100805098|1076496|1076.4960000000001|
|     7|  5277077|  5661667| 384590|384.58999999999997|
|     7|  1172124|  1526248| 354124|354.12400000000002|
|     8|144748397|145258668| 510271|510.27100000000002|
|     8|145286332|145682932| 396600|396.60000000000002|
|     9|130341832|132263422|1921590|1921.5899999999999|
|     9|135952412|137342684|1390272|1390.2719999999999|
|     9|134008327|134427749| 419422|419.42200000000003|
|     9|133754159|134008327| 254168|254.16800000000001|
|    11|   198510|  2330394|2131884|          21

In [16]:
val joinExpression = ctcDf.col("ctcChr") === df.col("chr")
val joinedDf = ctcDf.join(df, joinExpression).filter(col("ctcStart") <= col("end")).filter(col("start") <= col("ctcEnd"))
joinedDf.show

+------+---------+---------+-------+-------------------+---+---------+---------+----------+-----+-------+--------------+-------------------+-------------+----------------+------------+
|ctcChr| ctcStart|   ctcEnd| sizeBp|            sizeKbp|chr|    start|      end|copyNumber|event|   logR|subcloneStatus|CorrectedCopyNumber|correctedCall|  logRCopyNumber|sampleNumber|
+------+---------+---------+-------+-------------------+---+---------+---------+----------+-----+-------+--------------+-------------------+-------------+----------------+------------+
|     1|  1828508|  3571573|1743065| 1743.0650000000001|  1|  1000001|  2000000|         3| GAIN|     NA|             1|                  3|         GAIN|              NA|          35|
|     1|  1828508|  3571573|1743065| 1743.0650000000001|  1|  3000001|  4000000|         3| GAIN| 0.2532|             1|                  3|         GAIN|  4.669246525207|          35|
|     1|154963903|155224815| 260912| 260.91199999999998|  1|154000001|15500

joinExpression: org.apache.spark.sql.Column = (ctcChr = chr)
joinedDf: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [ctcChr: int, ctcStart: int ... 14 more fields]


In [17]:
joinedDf.count

res14: Long = 1278


In [19]:
joinedDf.coalesce(1).write.format("csv").mode("overwrite").option("header", true).option("sep", ",").save("/home/tomas/Documents/Martin/CTCRegions/output")